In [1]:
import os
os.chdir('../..')

In [2]:
import numpy as np
import pandas as pd
from collections import Iterable, defaultdict
import random
from os import listdir
from os.path import isfile, join
import json
import pickle

from itertools import combinations

import matplotlib.pyplot as plt

# For the Python notebook
%matplotlib inline
%reload_ext autoreload
%autoreload 2

D:\Users\glede\anaconda3\envs\py37\lib\site-packages\ipykernel_launcher.py:3: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated since Python 3.3,and in 3.9 it will stop working
  This is separate from the ipykernel package so we can avoid doing imports until


# Functions

In [3]:
def is_a_DATGAN(name):
    if any(x in name for x in ['TGAN', 'CTGAN', 'TVAE', 'FULL', 'TRANSRED', 'LINEAR', 'NOLINKS', 'PREDICTION']):
        return False
    else:
        return True

def compute_stats(freq_list_orig, freq_list_synth):
    """
    Different statistics computed on the frequency list
    
    """
    freq_list_orig, freq_list_synth = np.array(freq_list_orig), np.array(freq_list_synth)
    corr_mat = np.corrcoef(freq_list_orig, freq_list_synth)
    corr = corr_mat[0, 1]
    if np.isnan(corr): corr = 0.0
    # MAE
    mae = np.absolute(freq_list_orig - freq_list_synth).mean()
    # RMSE
    rmse = np.linalg.norm(freq_list_orig - freq_list_synth) / np.sqrt(len(freq_list_orig))
    # SRMSE
    freq_list_orig_avg = freq_list_orig.mean()
    srmse = rmse / freq_list_orig_avg
    # r-square
    u = np.sum((freq_list_synth - freq_list_orig)**2)
    v = np.sum((freq_list_orig - freq_list_orig_avg)**2)
    r2 = 1.0 - u / v
    stat = {'mae': mae, 'rmse': rmse, 'r2': r2, 'srmse': srmse, 'corr': corr}
    
    return stat

# Get all models and associated files

In [4]:
dataset = 'LPMC'
n_models = 5
n_data = 5

# Models for testing all DATGANS
models = ['CTGAN', 'TGAN', 'TVAE']

for i in ['WGAN', 'SGAN', 'WGGP']:
    for j in ['WI', 'OR', 'WO']:
        for k in ['NO', 'BO', 'OD', 'OC']:
            models.append('{}_{}_{}'.format(i,j,k))
            
# Models for testing different DAGs
if 'DAG' in dataset:
    models = ['FULL', 'TRANSRED', 'LINEAR', 'NOLINKS', 'PREDICTION']
            
models.sort()

files_ = {}

for m in models:
    tmp = []
    if is_a_DATGAN(m):
        spl = m.split('_')
        for i in range(n_models):
            for j in range(n_data):
                tmp.append('{}_{}_{:0>2}_{}_{:0>2}.csv'.format(spl[0], spl[1], i+1,  spl[2], j+1))
    else:
        for i in range(n_models):
            for j in range(n_data):
                tmp.append('{}_{:0>2}_{:0>2}.csv'.format(m, i+1, j+1))
    files_[m] = tmp


input_folder = '../synth_data/{}/'.format(dataset)

In [5]:
df_orig = pd.read_csv('../data/' + dataset.split('_')[0] + '/data.csv')

In [6]:
if 'Chicago' in dataset:
    continuous_cols = ['distance', 'age', 'departure_time']
elif 'LPMC' in dataset:
    continuous_cols = ['start_time_linear', 'age', 'distance', 'dur_walking', 'dur_cycling', 'dur_pt_access', 'dur_pt_rail', 'dur_pt_bus', 'dur_pt_int', 'dur_driving', 'cost_transit', 'cost_driving_fuel', 'driving_traffic_percent']

In [7]:
bins_cont = {}

for c in continuous_cols:
    #bins_cont[c] = pd.qcut(df_orig[c], q=10, retbins=True)[1]
    bins_cont[c] = pd.cut(df_orig[c], bins=10, retbins=True)[1]
    bins_cont[c][0] = -np.inf
    bins_cont[c][-1] = np.inf
    df_orig[c] = pd.cut(df_orig[c], bins=bins_cont[c])

In [8]:
df_orig.head()

,travel_mode,purpose,fueltype,faretype,bus_scale,travel_year,travel_month,travel_date,day_of_week,start_time_linear,...,dur_pt_access,dur_pt_rail,dur_pt_bus,dur_pt_int,pt_n_interchanges,dur_driving,cost_transit,cost_driving_fuel,cost_driving_con_charge,driving_traffic_percent
0,drive,HBO,Petrol_Car,child,0.0,2012,4,1,7,"(9.567, 11.958]",...,"(0.106, 0.212]","(-inf, 0.137]","(-inf, 0.215]","(-inf, 0.0567]",0,"(-inf, 0.183]","(-inf, 1.17]","(-inf, 1.027]",0.0,"(0.104, 0.208]"
1,drive,HBO,Petrol_Car,free,0.0,2012,4,1,7,"(16.742, 19.133]",...,"(0.212, 0.318]","(-inf, 0.137]","(-inf, 0.215]","(-inf, 0.0567]",0,"(-inf, 0.183]","(-inf, 1.17]","(-inf, 1.027]",0.0,"(-inf, 0.104]"
2,drive,HBO,Petrol_Car,full,1.0,2012,4,1,7,"(11.958, 14.35]",...,"(0.212, 0.318]","(-inf, 0.137]","(0.859, 1.074]","(0.0567, 0.113]",1,"(0.362, 0.54]","(2.34, 3.51]","(1.027, 2.034]",0.0,"(0.313, 0.417]"
3,pt,HBW,Average_Car,full,1.0,2012,4,1,7,"(19.133, 21.525]",...,"(0.106, 0.212]","(-inf, 0.137]","(-inf, 0.215]","(0.0567, 0.113]",1,"(-inf, 0.183]","(2.34, 3.51]","(-inf, 1.027]",0.0,"(-inf, 0.104]"
4,pt,HBO,Average_Car,free,0.0,2012,4,1,7,"(7.175, 9.567]",...,"(0.106, 0.212]","(-inf, 0.137]","(0.215, 0.429]","(-inf, 0.0567]",0,"(0.183, 0.362]","(-inf, 1.17]","(-inf, 1.027]",0.0,"(-inf, 0.104]"


In [9]:
stats_str = ['mae', 'rmse', 'r2', 'srmse', 'corr']
orig_str = 'random-original'

In [10]:
try:
    os.makedirs('./notebooks/results/{}'.format(dataset))
except:
    pass

# Stats per individual column

In [11]:
filepath = './notebooks/results/{}/'.format(dataset)
filename = 'single_columns.pickle'.format(dataset)

all_stats = {}

try:
    all_stats = pickle.load(open(filepath + filename, 'rb'))
    print('Found previous pickel file, using that')
except:
    print('No previous results found, starting fresh')
    try:
        os.makedirs(filepath)
    except:
        pass

Found previous pickel file, using that


In [12]:
# Go through each model
for i, m in enumerate(models):
    
    if m in all_stats:
        print("Results for model \033[1m{}\033[0m ({}/{}) already exists!".format(m, i+1, len(models)))

    else:
        print("Preparing stats for model \033[1m{}\033[0m ({}/{})".format(m, i+1, len(models)))

        all_stats[m] = {}

        for c in df_orig.columns:
            all_stats[m][c] = {}
            for s in stats_str:
                all_stats[m][c][s] = []

        # Load all dataframes for current model
        dfs = [pd.read_csv(input_folder + f) for f in files_[m]]

        # Go through all dataframes generated for each model
        for df in dfs:

            # Discretize continuous columns
            for c in continuous_cols:
                df[c] = pd.cut(df[c], bins=bins_cont[c])

            # Go through each columns
            for c in df_orig.columns:

                agg_vars = [c]

                real = df_orig.copy()
                real['count'] = 1
                real = real.groupby(agg_vars, observed=True).count()
                real /= len(df_orig)

                synth = df.copy()
                synth['count'] = 1
                synth = synth.groupby(agg_vars, observed=True).count()
                synth /= len(df)

                real_and_sampled = pd.merge(real, synth, suffixes=['_real', '_sampled'], on=agg_vars, how='outer', indicator=True)
                real_and_sampled = real_and_sampled[['count_real', 'count_sampled']].fillna(0)

                sts = compute_stats(real_and_sampled['count_real'], real_and_sampled['count_sampled'])

                for s in sts:
                    all_stats[m][c][s].append(sts[s])
                    
        pickle.dump(all_stats, open(filepath + filename, 'wb'))

print("\033[1mFINISHED!\033[0m")

Results for model CTGAN (1/39) already exists!
Results for model SGAN_OR_BO (2/39) already exists!
Results for model SGAN_OR_NO (3/39) already exists!
Results for model SGAN_OR_OC (4/39) already exists!
Results for model SGAN_OR_OD (5/39) already exists!
Results for model SGAN_WI_BO (6/39) already exists!
Results for model SGAN_WI_NO (7/39) already exists!
Results for model SGAN_WI_OC (8/39) already exists!
Results for model SGAN_WI_OD (9/39) already exists!
Results for model SGAN_WO_BO (10/39) already exists!
Results for model SGAN_WO_NO (11/39) already exists!
Results for model SGAN_WO_OC (12/39) already exists!
Results for model SGAN_WO_OD (13/39) already exists!
Results for model TGAN (14/39) already exists!
Results for model TVAE (15/39) already exists!
Results for model WGAN_OR_BO (16/39) already exists!
Results for model WGAN_OR_NO (17/39) already exists!
Results for model WGAN_OR_OC (18/39) already exists!
Results for model WGAN_OR_OD (19/39) already exists!
Results for model W

In [13]:
if orig_str not in all_stats:

    stats_orig = {}

    for c in df_orig.columns:
        stats_orig[c] = {}
        for s in stats_str:
            stats_orig[c][s] = []

    for i in range(n_models*n_data):

        train = df_orig.sample(int(len(df_orig) * 0.5))
        train.index = range(len(train))
        test = df_orig[~df_orig.index.isin(train.index)]
        test.index = range(len(test))

        # Go through each columns
        for c in df_orig.columns:

            agg_vars = [c]

            real = train.copy()
            real['count'] = 1
            real = real.groupby(agg_vars, observed=True).count()
            real /= len(df_orig)

            synth = test.copy()
            synth['count'] = 1
            synth = synth.groupby(agg_vars, observed=True).count()
            synth /= len(df)

            real_and_sampled = pd.merge(real, synth, suffixes=['_real', '_sampled'], on=agg_vars, how='outer', indicator=True)
            real_and_sampled = real_and_sampled[['count_real', 'count_sampled']].fillna(0)

            sts = compute_stats(real_and_sampled['count_real'], real_and_sampled['count_sampled'])

            for s in sts:
                stats_orig[c][s].append(sts[s])
    
    all_stats[orig_str] = stats_orig
    
    pickle.dump(all_stats, open(filepath + filename, 'wb'))

In [14]:
res = {}

for test in ['all', 'cont', 'cat']:
    
    res[test] = {}
    
    if test == 'all':
        cols = df_orig.columns
    elif test == 'cont':
        cols = continuous_cols
    elif test == 'cat':
        cols = set(df_orig.columns) - set(continuous_cols)

    for s in stats_str:
        res[test][s] = {}

    for m in all_stats.keys():

        for s in stats_str:
            res[test][s][m] = []

            for i in range(n_models*n_data):
                tmp = []

                for c in cols:
                    tmp.append(all_stats[m][c][s][i])

                res[test][s][m].append(np.mean(tmp))

In [15]:
avg = {}

for test in ['all', 'cont', 'cat']:
    
    avg[test] = {}

    for s in stats_str:
        avg[test][s] = {}

        for m in all_stats.keys():
            avg[test][s][m] = {
                'mean': np.mean(res[test][s][m]),
                'std': np.std(res[test][s][m])
            }

In [16]:
for test in ['all', 'cont', 'cat']:
    
    if test == 'all':
        str_ = 'on all columns'
    elif test == 'cont':
        str_ = 'on continuous columns'
    elif test == 'cat':
        str_ = 'on categorical columns'
        
    for s in ['srmse']:#stats:
        print('Ranking {} based on {}:'.format(str_, s.upper()))

        if s in ['r2', 'corr']:
            sorted_dct = {k: v for k, v in sorted(avg[test][s].items(), key=lambda item: item[1]['mean'])[::-1]}
        else:
            sorted_dct = {k: v for k, v in sorted(avg[test][s].items(), key=lambda item: item[1]['mean'])}

        for i, item in enumerate(sorted_dct):
            print('  {:>2}. {:<15} - {:.2e} ± {:.2e}'.format(i+1, item, sorted_dct[item]['mean'], sorted_dct[item]['std']))
        print()


Ranking on all columns based on SRMSE:
   1. random-original - 7.50e-02 ± 4.56e-03
   2. WGGP_WI_NO      - 8.13e-02 ± 6.37e-03
   3. WGGP_WI_OC      - 8.77e-02 ± 6.71e-03
   4. SGAN_WI_NO      - 8.99e-02 ± 4.44e-03
   5. SGAN_WI_OC      - 9.24e-02 ± 4.40e-03
   6. WGGP_WO_OC      - 9.53e-02 ± 8.33e-03
   7. SGAN_WI_OD      - 9.75e-02 ± 5.94e-03
   8. WGGP_WI_OD      - 9.77e-02 ± 6.90e-03
   9. WGGP_WO_NO      - 1.00e-01 ± 5.68e-03
  10. SGAN_WI_BO      - 1.00e-01 ± 8.56e-03
  11. WGGP_WO_BO      - 1.00e-01 ± 8.35e-03
  12. WGGP_WI_BO      - 1.04e-01 ± 6.56e-03
  13. WGGP_WO_OD      - 1.05e-01 ± 6.28e-03
  14. WGAN_WI_NO      - 1.12e-01 ± 5.69e-03
  15. SGAN_WO_NO      - 1.13e-01 ± 1.08e-02
  16. WGGP_OR_OD      - 1.15e-01 ± 1.03e-02
  17. WGAN_WI_OC      - 1.18e-01 ± 7.46e-03
  18. WGGP_OR_BO      - 1.20e-01 ± 8.12e-03
  19. SGAN_WO_OC      - 1.22e-01 ± 9.72e-03
  20. SGAN_WO_OD      - 1.27e-01 ± 1.26e-02
  21. SGAN_OR_OD      - 1.27e-01 ± 1.94e-02
  22. WGAN_WO_NO      - 1.33e-01 ± 1.

# Stats per couple columns

In [17]:
combs = []

for k in combinations(df_orig.columns, 2):
    combs.append(k[0] + '::' + k[1])

In [18]:
filepath = './notebooks/results/{}/'.format(dataset)
filename = 'couple_combinations.pickle'.format(dataset)

all_stats = {}

try:
    all_stats = pickle.load(open(filepath + filename, 'rb'))
    print('Found previous pickel file, using that')
except:
    print('No previous results found, starting fresh')
    try:
        os.makedirs(filepath)
    except:
        pass

Found previous pickel file, using that


In [19]:
# Go through each model
for i, m in enumerate(models):
    
    if m in all_stats:
        print("Results for model \033[1m{}\033[0m ({}/{}) already exists!".format(m, i+1, len(models)))

    else:
        print("Preparing stats for model \033[1m{}\033[0m ({}/{})".format(m, i+1, len(models)))

        all_stats[m] = {}

        for c in combs:
            all_stats[m][c] = {}
            for s in stats_str:
                all_stats[m][c][s] = []

        # Load all dataframes for current model
        dfs = [pd.read_csv(input_folder + f) for f in files_[m]]

        # Go through all dataframes generated for each model
        for df in dfs:

            # Discretize continuous columns
            for c in continuous_cols:
                df[c] = pd.cut(df[c], bins=bins_cont[c])

            # Go through each columns
            for c in combs:

                agg_vars = c.split('::')

                real = df_orig.copy()
                real['count'] = 1
                real = real.groupby(agg_vars, observed=True).count()
                real /= len(df_orig)

                synth = df.copy()
                synth['count'] = 1
                synth = synth.groupby(agg_vars, observed=True).count()
                synth /= len(df)

                real_and_sampled = pd.merge(real, synth, suffixes=['_real', '_sampled'], on=agg_vars, how='outer', indicator=True)
                real_and_sampled = real_and_sampled[['count_real', 'count_sampled']].fillna(0)

                sts = compute_stats(real_and_sampled['count_real'], real_and_sampled['count_sampled'])

                for s in sts:
                    all_stats[m][c][s].append(sts[s])
                    
        pickle.dump(all_stats, open(filepath + filename, 'wb'))

print("\033[1mFINISHED!\033[0m")

Results for model CTGAN (1/39) already exists!
Results for model SGAN_OR_BO (2/39) already exists!
Results for model SGAN_OR_NO (3/39) already exists!
Results for model SGAN_OR_OC (4/39) already exists!
Results for model SGAN_OR_OD (5/39) already exists!
Results for model SGAN_WI_BO (6/39) already exists!
Results for model SGAN_WI_NO (7/39) already exists!
Results for model SGAN_WI_OC (8/39) already exists!
Results for model SGAN_WI_OD (9/39) already exists!
Results for model SGAN_WO_BO (10/39) already exists!
Results for model SGAN_WO_NO (11/39) already exists!
Results for model SGAN_WO_OC (12/39) already exists!
Results for model SGAN_WO_OD (13/39) already exists!
Results for model TGAN (14/39) already exists!
Results for model TVAE (15/39) already exists!
Results for model WGAN_OR_BO (16/39) already exists!
Results for model WGAN_OR_NO (17/39) already exists!
Results for model WGAN_OR_OC (18/39) already exists!
Results for model WGAN_OR_OD (19/39) already exists!
Results for model W

In [20]:
if orig_str not in all_stats:
    stats_orig = {}

    for c in combs:
        stats_orig[c] = {}
        for s in stats_str:
            stats_orig[c][s] = []

    for i in range(n_models*n_data):

        train = df_orig.sample(int(len(df_orig) * 0.5))
        train.index = range(len(train))
        test = df_orig[~df_orig.index.isin(train.index)]
        test.index = range(len(test))

        # Go through each columns
        for c in combs:

            agg_vars = c.split('::')

            real = train.copy()
            real['count'] = 1
            real = real.groupby(agg_vars, observed=True).count()
            real /= len(df_orig)

            synth = test.copy()
            synth['count'] = 1
            synth = synth.groupby(agg_vars, observed=True).count()
            synth /= len(df)

            real_and_sampled = pd.merge(real, synth, suffixes=['_real', '_sampled'], on=agg_vars, how='outer', indicator=True)
            real_and_sampled = real_and_sampled[['count_real', 'count_sampled']].fillna(0)

            sts = compute_stats(real_and_sampled['count_real'], real_and_sampled['count_sampled'])

            for s in sts:
                stats_orig[c][s].append(sts[s])
                
    all_stats[orig_str] = stats_orig
    
    pickle.dump(all_stats, open(filepath + filename, 'wb'))

In [21]:
res = {}

for s in stats_str:
    res[s] = {}

for m in all_stats.keys():

    for s in stats_str:
        res[s][m] = []

        for i in range(n_models*n_data):
            tmp = []

            for c in combs:
                tmp.append(all_stats[m][c][s][i])

            res[s][m].append(np.mean(tmp))

In [22]:
avg = {}

for s in stats_str:
    avg[s] = {}

    for m in all_stats.keys():
        avg[s][m] = {
            'mean': np.mean(res[s][m]),
            'std': np.std(res[s][m])
        }

In [23]:
for s in ['srmse']:#stats:
    print('Ranking on all coupled combinations based on {}:'.format(s.upper()))

    if s in ['r2', 'corr']:
        sorted_dct = {k: v for k, v in sorted(avg[s].items(), key=lambda item: item[1]['mean'])[::-1]}
    else:
        sorted_dct = {k: v for k, v in sorted(avg[s].items(), key=lambda item: item[1]['mean'])}

    for i, item in enumerate(sorted_dct):
        print('  {:>2}. {:<15} - {:.2e} ± {:.2e}'.format(i+1, item, sorted_dct[item]['mean'], sorted_dct[item]['std']))
    print()


Ranking on all coupled combinations based on SRMSE:
   1. random-original - 2.02e-01 ± 4.69e-03
   2. WGGP_WI_NO      - 2.26e-01 ± 9.03e-03
   3. WGGP_WI_OC      - 2.35e-01 ± 1.30e-02
   4. WGGP_WI_OD      - 2.71e-01 ± 1.25e-02
   5. WGGP_WO_OC      - 2.75e-01 ± 1.18e-02
   6. WGGP_WO_NO      - 2.77e-01 ± 1.69e-02
   7. WGGP_WI_BO      - 2.77e-01 ± 1.69e-02
   8. SGAN_WI_NO      - 2.78e-01 ± 1.67e-02
   9. SGAN_WI_OC      - 2.79e-01 ± 1.05e-02
  10. SGAN_WI_OD      - 2.82e-01 ± 1.63e-02
  11. WGGP_WO_BO      - 2.85e-01 ± 1.36e-02
  12. SGAN_WI_BO      - 2.85e-01 ± 1.48e-02
  13. WGGP_WO_OD      - 2.86e-01 ± 1.63e-02
  14. WGGP_OR_OD      - 2.88e-01 ± 1.64e-02
  15. WGGP_OR_BO      - 2.97e-01 ± 2.12e-02
  16. SGAN_WO_NO      - 3.27e-01 ± 3.07e-02
  17. WGAN_WI_OC      - 3.28e-01 ± 1.29e-02
  18. WGAN_WI_NO      - 3.33e-01 ± 7.08e-03
  19. SGAN_WO_OC      - 3.37e-01 ± 2.93e-02
  20. SGAN_WO_OD      - 3.38e-01 ± 3.33e-02
  21. SGAN_OR_OD      - 3.48e-01 ± 4.54e-02
  22. SGAN_WO_BO      - 

# Stats per trouple columns

In [24]:
combs = []

for k in combinations(df_orig.columns, 3):
    combs.append(k[0] + '::' + k[1] + '::' + k[2])

In [25]:
filepath = './notebooks/results/{}/'.format(dataset)
filename = 'trouple_combinations.pickle'.format(dataset)

all_stats = {}

try:
    all_stats = pickle.load(open(filepath + filename, 'rb'))
    print('Found previous pickel file, using that')
except:
    print('No previous results found, starting fresh')
    try:
        os.makedirs(filepath)
    except:
        pass

Found previous pickel file, using that


In [26]:
# Go through each model
for i, m in enumerate(models):

    if m in all_stats:
        print("Results for model \033[1m{}\033[0m ({}/{}) already exists!".format(m, i+1, len(models)))
    else:
        print("Preparing stats for model \033[1m{}\033[0m ({}/{})".format(m, i+1, len(models)))

        all_stats[m] = {}

        for c in combs:
            all_stats[m][c] = {}
            for s in stats_str:
                all_stats[m][c][s] = []

        # Load all dataframes for current model
        dfs = [pd.read_csv(input_folder + f) for f in files_[m]]

        # Go through all dataframes generated for each model
        for df in dfs:

            # Discretize continuous columns
            for c in continuous_cols:
                df[c] = pd.cut(df[c], bins=bins_cont[c])

            # Go through each columns
            for c in combs:

                agg_vars = c.split('::')

                real = df_orig.copy()
                real['count'] = 1
                real = real.groupby(agg_vars, observed=True).count()
                real /= len(df_orig)

                synth = df.copy()
                synth['count'] = 1
                synth = synth.groupby(agg_vars, observed=True).count()
                synth /= len(df)

                real_and_sampled = pd.merge(real, synth, suffixes=['_real', '_sampled'], on=agg_vars, how='outer', indicator=True)
                real_and_sampled = real_and_sampled[['count_real', 'count_sampled']].fillna(0)

                sts = compute_stats(real_and_sampled['count_real'], real_and_sampled['count_sampled'])

                for s in sts:
                    all_stats[m][c][s].append(sts[s])
    
        pickle.dump(all_stats, open(filepath + filename, 'wb'))

print("\033[1mFINISHED!\033[0m")

Results for model CTGAN (1/39) already exists!
Results for model SGAN_OR_BO (2/39) already exists!
Results for model SGAN_OR_NO (3/39) already exists!
Results for model SGAN_OR_OC (4/39) already exists!
Results for model SGAN_OR_OD (5/39) already exists!
Results for model SGAN_WI_BO (6/39) already exists!
Results for model SGAN_WI_NO (7/39) already exists!
Results for model SGAN_WI_OC (8/39) already exists!
Results for model SGAN_WI_OD (9/39) already exists!
Results for model SGAN_WO_BO (10/39) already exists!
Results for model SGAN_WO_NO (11/39) already exists!
Results for model SGAN_WO_OC (12/39) already exists!
Results for model SGAN_WO_OD (13/39) already exists!
Results for model TGAN (14/39) already exists!
Results for model TVAE (15/39) already exists!
Results for model WGAN_OR_BO (16/39) already exists!
Results for model WGAN_OR_NO (17/39) already exists!
Results for model WGAN_OR_OC (18/39) already exists!
Results for model WGAN_OR_OD (19/39) already exists!
Results for model W

In [27]:
if orig_str not in all_stats:
    stats_orig = {}

    for c in combs:
        stats_orig[c] = {}
        for s in stats_str:
            stats_orig[c][s] = []

    for i in range(n_models*n_data):

        train = df_orig.sample(int(len(df_orig) * 0.5))
        train.index = range(len(train))
        test = df_orig[~df_orig.index.isin(train.index)]
        test.index = range(len(test))

        # Go through each columns
        for c in combs:

            agg_vars = c.split('::')

            real = train.copy()
            real['count'] = 1
            real = real.groupby(agg_vars, observed=True).count()
            real /= len(df_orig)

            synth = test.copy()
            synth['count'] = 1
            synth = synth.groupby(agg_vars, observed=True).count()
            synth /= len(df)

            real_and_sampled = pd.merge(real, synth, suffixes=['_real', '_sampled'], on=agg_vars, how='outer', indicator=True)
            real_and_sampled = real_and_sampled[['count_real', 'count_sampled']].fillna(0)

            sts = compute_stats(real_and_sampled['count_real'], real_and_sampled['count_sampled'])

            for s in sts:
                stats_orig[c][s].append(sts[s])
                
    all_stats[orig_str] = stats_orig
    
    pickle.dump(all_stats, open(filepath + filename, 'wb'))

In [28]:
res = {}

for s in stats_str:
    res[s] = {}

for m in all_stats.keys():

    for s in stats_str:
        res[s][m] = []

        for i in range(n_models*n_data):
            tmp = []

            for c in combs:
                tmp.append(all_stats[m][c][s][i])

            res[s][m].append(np.mean(tmp))

In [29]:
avg = {}

for s in stats_str:
    avg[s] = {}

    for m in all_stats.keys():
        avg[s][m] = {
            'mean': np.mean(res[s][m]),
            'std': np.std(res[s][m])
        }

In [30]:
for s in ['srmse']:#stats_str:
    print('Ranking on all triple combinations based on {}:'.format(s.upper()))

    if s in ['r2', 'corr']:
        sorted_dct = {k: v for k, v in sorted(avg[s].items(), key=lambda item: item[1]['mean'])[::-1]}
    else:
        sorted_dct = {k: v for k, v in sorted(avg[s].items(), key=lambda item: item[1]['mean'])}

    for i, item in enumerate(sorted_dct):
        print('  {:>2}. {:<15} - {:.2e} ± {:.2e}'.format(i+1, item, sorted_dct[item]['mean'], sorted_dct[item]['std']))
    print()


Ranking on all triple combinations based on SRMSE:
   1. random-original - 3.91e-01 ± 8.54e-03
   2. WGGP_WI_NO      - 4.50e-01 ± 2.00e-02
   3. WGGP_WI_OC      - 4.61e-01 ± 2.51e-02
   4. WGGP_WI_OD      - 5.33e-01 ± 2.59e-02
   5. WGGP_WO_OC      - 5.36e-01 ± 2.82e-02
   6. WGGP_WO_NO      - 5.37e-01 ± 3.14e-02
   7. WGGP_WI_BO      - 5.39e-01 ± 3.11e-02
   8. WGGP_OR_OD      - 5.42e-01 ± 3.12e-02
   9. WGGP_OR_BO      - 5.44e-01 ± 2.79e-02
  10. WGGP_WO_BO      - 5.49e-01 ± 3.32e-02
  11. WGGP_WO_OD      - 5.50e-01 ± 3.14e-02
  12. SGAN_WI_OD      - 5.55e-01 ± 3.48e-02
  13. SGAN_WI_OC      - 5.60e-01 ± 2.29e-02
  14. SGAN_WI_NO      - 5.60e-01 ± 3.52e-02
  15. SGAN_WI_BO      - 5.62e-01 ± 2.76e-02
  16. WGAN_WI_OC      - 6.22e-01 ± 1.67e-02
  17. SGAN_WO_NO      - 6.32e-01 ± 5.95e-02
  18. SGAN_WO_OD      - 6.33e-01 ± 6.64e-02
  19. SGAN_WO_OC      - 6.37e-01 ± 5.65e-02
  20. WGAN_WI_NO      - 6.47e-01 ± 8.48e-03
  21. SGAN_WO_BO      - 6.48e-01 ± 6.56e-02
  22. SGAN_OR_OD      - 6